In [3]:
!pip install --upgrade transformers
!pip install datasets


In [4]:
from datasets import load_dataset

# Load a small portion (first 2000 samples) of the "helpful-base" split
dataset = load_dataset("Anthropic/hh-rlhf", data_dir="helpful-base")
small_dataset = dataset['train'].select(range(1000))


from datasets import load_dataset

ds = load_dataset("harouzie/vi_en-translation", split="train")
print(ds[0])  # in phần tử đầu tiên


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


{'English': "I heard that since Tom isn't feeling well he won't come to school today", 'Vietnamese': 'tôi nghe nói rằng vì tom không khỏe nên hôm nay anh ấy sẽ không đến trường'}


In [2]:
!pip install trl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
!pip install trlfrom datasets import load_dataset
from transformers import MarianTokenizer, MarianMTModel, Seq2SeqTrainer, Seq2SeqTrainingArguments

# Load only the first 2000 rows from the dataset
ds = load_dataset("harouzie/vi_en-translation", split='train[:1000]')

# Load tokenizer & model
model_name = "Helsinki-NLP/opus-mt-en-vi"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Preprocessing function
def preprocess(examples):
    src_texts = examples['English']
    tgt_texts = examples['Vietnamese']

    model_inputs = tokenizer(src_texts, truncation=True, padding='max_length', max_length=128)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(tgt_texts, truncation=True, padding='max_length', max_length=128)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply preprocessing
tokenized_ds = ds.map(preprocess, batched=True)

# Training configuration
training_args = Seq2SeqTrainingArguments(
    output_dir="./mt-model",
    per_device_train_batch_size=8,
    num_train_epochs=3,
    save_steps=500,
    logging_steps=100,
)

# Trainer setup
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds,
    tokenizer=tokenizer,
)

# Start training
trainer.train()

trainer.save_model("./mt-model")
tokenizer.save_pretrained("./mt-model")

ERROR: Could not find a version that satisfies the requirement trlfrom (from versions: none)
ERROR: No matching distribution found for trlfrom


/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
<ipython-input-5-e80323e37e60>:38: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

...

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: gianghoangdat2003 (gianghoangdat2003-sada) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
100,0.248700
200,0.038600
300,0.028900


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[53684]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


('./mt-model/tokenizer_config.json',
 './mt-model/special_tokens_map.json',
 './mt-model/vocab.json',
 './mt-model/source.spm',
 './mt-model/target.spm',
 './mt-model/added_tokens.json')

In [ ]:
from trl import PPOTrainer, PPOConfig

human_feedback = [
    {"input": "Hello, how are you?", "prediction": "Xin chào, bạn khỏe không?", "feedback": 1},
    {"input": "I am learning machine translation.", "prediction": "Tôi đang học dịch máy.", "feedback": 0},
    {"input": "This is a test sentence.", "prediction": "Đây là một câu thử nghiệm.", "feedback": 1},
]

# 8. Cấu hình PPO cho RLHF
ppo_config = PPOConfig(
    model_name="Helsinki-NLP/opus-mt-en-vi",  # Tên model đã huấn luyện
    batch_size=8,
    forward_batch_size=4,
    num_train_epochs=3,
    learning_rate=5e-5,
    logging_steps=100,
    save_steps=500,
    output_dir="./ppo_model"
)

# 9. Tạo PPOTrainer
ppo_trainer = PPOTrainer(
    config=ppo_config,
    model=model,  # Mô hình Seq2Seq đã huấn luyện
    tokenizer=tokenizer
)

# 10. Tạo tập huấn luyện từ phản hồi người dùng
train_data = []
for item in human_feedback:
    input_text = item["input"]
    prediction = item["prediction"]
    reward = item["feedback"]
    train_data.append((input_text, prediction, reward))

# 11. Huấn luyện với PPO
for input_text, prediction, reward in train_data:
    ppo_trainer.train_step(input_text, prediction, reward)

# Lưu mô hình PPO đã huấn luyện
ppo_trainer.save_model("./ppo_model")
tokenizer.save_pretrained("./ppo_model")

# 12. Dự đoán sau khi huấn luyện với RLHF
input_text = "How is the weather today?"
encoded_input = tokenizer(input_text, return_tensors="pt")
output = model.generate(**encoded_input)
decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

# In kết quả dự đoán
print(f"Dự đoán của mô hình: {decoded_output}")

In [28]:
query = "He is working hard to support his family."
input_ids = tokenizer(query, return_tensors="pt", truncation=True, padding=True).input_ids

# Sinh bản dịch
translated_ids = model.generate(input_ids)
translated_text = tokenizer.batch_decode(translated_ids, skip_special_tokens=True)[0]

# Đánh giá bản dịch -> reward
reward = reward_model(translated_text)[0]['score']

# Huấn luyện với PPO
ppo_trainer.step([query], [translated_text], [reward])


Help on class PPOConfig in module trl.trainer.ppo_config:

class PPOConfig(trl.trainer.utils.OnPolicyConfig)
 |  PPOConfig(output_dir: Optional[str] = None, overwrite_output_dir: bool = False, do_train: bool = False, do_eval: bool = False, do_predict: bool = False, eval_strategy: Union[transformers.trainer_utils.IntervalStrategy, str] = 'no', prediction_loss_only: bool = False, per_device_train_batch_size: int = 8, per_device_eval_batch_size: int = 8, per_gpu_train_batch_size: Optional[int] = None, per_gpu_eval_batch_size: Optional[int] = None, gradient_accumulation_steps: int = 1, eval_accumulation_steps: Optional[int] = None, eval_delay: Optional[float] = 0, torch_empty_cache_steps: Optional[int] = None, learning_rate: float = 5e-05, weight_decay: float = 0.0, adam_beta1: float = 0.9, adam_beta2: float = 0.999, adam_epsilon: float = 1e-08, max_grad_norm: float = 1.0, num_train_epochs: float = 3.0, max_steps: int = -1, lr_scheduler_type: Union[transformers.trainer_utils.SchedulerType,